We have a problem. We don't have normal input for our problem and check something "by hands" is also complicated, because of problem is too hard to get an analytic solutuion. Sorry :(

In [2]:
import numpy as np
import scipy.stats as sps
from cvxopt import solvers, matrix

In [56]:
n = 2 # number of points
A = np.diag(np.zeros(n) + 1) + np.diag(np.zeros(n - 1) + 1, k=-1)
theta = np.pi / 2 # given theta_1, other theta_i comes our from theta_1

In [57]:
l = np.array([1, 1])
b = np.array([theta, np.pi])
x_start = np.linalg.inv(A) @ b

array([1.57079633, 3.14159265])

In [36]:
l = sps.uniform(loc=1, scale=10).rvs(size=10) # distances between a_i
b = np.concatenate((np.array([theta]), np.array(sps.uniform(loc=-np.pi, scale=2 * np.pi).rvs(size=9)))) # matrix of theta_1 and \psi_i
x_start = np.linalg.inv(A) @ b

Our real function to minimize and it's derivative. We drop the first coordinate because $\psi_1$ doesn't defined. Derivative of F in zero we define as a limit of $\frac{\sin x}{x} = 1$.

In [77]:
def F(x, l):
    x = x[1:]
    l = l[1:]
    return np.sum(l * x / np.sin(x))

def dF(x, l):
    x = x[1:]
    l = l[1:]
    return np.concatenate(([0], l * (np.sin(x) - x * np.cos(x)) / np.sin(x) ** 2))

def hessian(x, l):
    x = x[1:]
    l = l[1:]
    return np.diag(np.concatenate(([0], l * (1 + 2 * x * np.cos(x) ** 2) / np.sin(x) ** 3)))

Function from cvxopt

In [6]:
def acent(A, b):
    return solvers.cp(F, A=A, b=b)['x']

In [78]:
def gradient_descent(x_start, eps=1e-4, alpha = 1e-2):
    step_counter = 0
    x_cur = x_start
    x_new = x_cur - alpha * dF(x_cur, l)
    while np.abs(np.linalg.norm(x_cur - x_new)) > eps and np.linalg.norm(dF(x_new, l)) > eps:
        step_counter += 1
        if step_counter > 10000:
            raise Exception('For eps = %f and alpha = %f method diverges :('%(eps, alpha))
        if np.min(x_cur) < -np.pi or np.max(x_cur) > np.pi:
            raise Exception('For eps = %f and alpha = %f method comes out from the definition area:('%(eps, alpha))
        x_cur = x_new
        x_new = x_new = x_cur - alpha * dF(x_cur, l)
    print('Optimal solution for eps = %f, alpha = %f founded'%( eps, alpha))
    print('x_optimal =',(x_cur))

In [71]:
def newton_method(x_start, eps=1, alpha=1):
    step_counter = 0
    x_cur = x_start
    x_new = x_cur - alpha * F(x_cur, l) / dF(x_cur, l)
    while np.abs(np.linalg.norm(x_cur - x_new)) > eps and np.linalg.norm(F(x_new, l)) > eps:
        step_counter += 1
        if step_counter > 10000:
            raise Exception('For eps = %f and alpha = %f method diverges :('%(eps, alpha))
        if np.min(x_cur) < -np.pi or np.max(x_cur) > np.pi:
            raise Exception('For eps = %f and alpha = %f method comes out from the definition area:('%(eps, alpha))
        x_cur = x_new
        x_new = x_cur - alpha * F(x_cur, l) / dF(x_cur, l)
    print('Optimal solution for eps = %f, alpha = %f founded'%( eps, alpha))
    print('x_optimal =',(x_cur))

In [85]:
gradient_descent(x_start, 1e-10, 1e-1)
newton_method(x_start, 1e-10, 1e-2)

Optimal solution for eps = 0.000000, alpha = 0.100000 founded
x_optimal = [1.57079633e+00 2.26893082e-08]
Optimal solution for eps = 0.000000, alpha = 0.010000 founded
x_optimal = [      -inf 1.55508836]


/home/yanskii/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
/home/yanskii/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in true_divide
  if sys.path[0] == '':
/home/yanskii/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in subtract
  """


On normal input and functions method works.

In [11]:
def F(x):
    return np.cos(x) - x ** 3

def dF(x):
    return -np.sin(x) - 3 * x ** 2

x_start = 0.5

In [12]:
def F(x):
    return (x + 5) ** 2

def dF(x):
    return 2 * (x + 5)

x_start = 3

In [20]:
def test_gradient_descent(x_start, eps=1e-4, alpha = 1e-2):
    step_counter = 0
    x_cur = x_start
    x_new = x_cur - alpha * dF(x_cur)
    while np.abs(x_cur - x_new) > eps and np.abs(F(x_new) - F(x_cur)) > eps:
        step_counter += 1
        if step_counter > 10000:
            raise Exception('For eps = %f and alpha = %f method diverges :('%(eps, alpha))
        x_cur = x_new
        x_new = x_new = x_cur - alpha * dF(x_cur)
    print('Optimal solution for eps = %f, alpha = %f founded'%( eps, alpha))
    print('x_optimal =',(x_cur))

In [18]:
def test_newton_method(x_start, eps=1e-10, alpha=1):
    step_counter = 0
    x_cur = x_start
    x_new = x_cur - F(x_cur) / dF(x_cur)
    while np.abs(np.abs(x_cur - x_new)) > eps and np.linalg.norm(F(x_new)) > eps:
        step_counter += 1
        if step_counter > 10000:
            raise Exception('For eps = %f and alpha = %f method diverges :('%(eps, alpha))
        x_cur = x_new
        x_new = x_cur - F(x_cur) / dF(x_cur)
    print('Optimal solution for eps = %f, alpha = %f founded'%( eps, alpha))
    print('x_optimal =',(x_cur))

In [23]:
test_gradient_descent(x_start, 1e-4, 1e-2)
test_newton_method(x_start, 1e-10, 1)

Optimal solution for eps = 0.000100, alpha = 0.010000 founded
x_optimal = -4.949784824391848
Optimal solution for eps = 0.000000, alpha = 1.000000 founded
x_optimal = -4.9999847412109375


In [17]:
def hessian_matrix(x, l):
    return np.array([])